In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Read the generated csv
df = pd.read_csv('/content/drive/MyDrive/IS450 Group 5/zara/articles_with_dominant_topic.csv')
df.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label,dominant_topic,sentiment,frames,entities,political_bias,bias_probabilities
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,Tij Iginla doesn't shy away from his famous la...,images/39e6104d56.jpeg,Likely to be Bias,Likely to be Unbiased,0,0.148345,[],"['Tij Iginla', '17-year-old', ""Jarome Iginla's...",Right,"[0.4287033677101135, 0.08208581805229187, 0.48..."
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,This week on The Sunday Magazine with host Piy...,images/89ce50166b.jpeg,Likely to be Bias,Likely to be Unbiased,0,0.053554,"['security', 'political']","['This week', 'The Sunday Magazine', 'Piya Cha...",Left,"[0.782128632068634, 0.12677131593227386, 0.091..."
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,An Ontario Conservative MP's use of ChatGPT to...,images/8b320e107e.jpeg,Likely to be Bias,Likely to be Bias,0,0.020784,"['economic', 'political']","['Canada', 'AI', 'one', 'Ryan Williams', 'last...",Center,"[0.2644619345664978, 0.6984443068504333, 0.037..."
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,"When it comes to the study of sports, a man's ...",images/7536f87654.jpeg,Likely to be Unbiased,Likely to be Unbiased,0,0.169423,[],"['Institut', 'Québec', 'Égale Action', 'Sylvie...",Center,"[0.01947527751326561, 0.9609658122062683, 0.01..."
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,"To wrap up the 48th season of Quirks & Quarks,...",images/c829d1f9a8.jpeg,Likely to be Bias,Likely to be Unbiased,0,0.163121,['political'],"['the 48th season', 'Quirks & Quarks', 'Listen...",Center,"[0.01578383706510067, 0.9709484577178955, 0.01..."


In [ ]:
# Rename topic -1 to "Uncategorized"
df['dominant_topic'] = df['dominant_topic'].replace(-1, "Uncategorized")

# Verify the change
df['dominant_topic'].value_counts()

,count
dominant_topic,
0,37643
1,589
6,501
27,152
41,140
29,137
42,126
10,109
2,102


In [ ]:
pip install textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 9.1 MB/s eta 0:00:00


In [ ]:
import plotly.express as px
from textblob import TextBlob

# Sentiment analysis
df['sentiment'] = df['article_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Group by dominant topic and calculate average sentiment
topic_sentiment = df.groupby('dominant_topic')['sentiment'].mean().reset_index()

# Create an interactive bar plot using Plotly
fig = px.bar(topic_sentiment, x='dominant_topic', y='sentiment',
             title="Average Sentiment by Topic",
             labels={'dominant_topic': 'Topic', 'sentiment': 'Average Sentiment'},
             color='sentiment', color_continuous_scale='RdYlGn')

# Show the plot
fig.show()

In [ ]:
# Group by 'outlet' and 'dominant_topic' to calculate average sentiment
outlet_topic_sentiment = df.groupby(['outlet', 'dominant_topic'])['sentiment'].mean().reset_index()

# Create an interactive bar plot using Plotly
fig = px.bar(outlet_topic_sentiment, x='dominant_topic', y='sentiment', color='outlet',
             title="Average Sentiment by Topic and News Channel",
             labels={'dominant_topic': 'Topic', 'sentiment': 'Average Sentiment', 'outlet': 'News Channel'},
             color_discrete_sequence=px.colors.qualitative.Set1)

# Show the plot
fig.show()

In [ ]:
import plotly.express as px
from textblob import TextBlob

# Function to map sentiment score to description
def sentiment_description(score):
    if score >= 0.6:
        return "Extremely Positive"
    elif score >= 0.2:
        return "Positive"
    elif score > -0.2:
        return "Neutral"
    elif score > -0.6:
        return "Negative"
    else:
        return "Extremely Negative"

# Sentiment analysis
df['sentiment'] = df['article_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Add sentiment description for each article
df['sentiment_label'] = df['sentiment'].apply(sentiment_description)

# Group by dominant topic and calculate average sentiment
topic_sentiment = df.groupby('dominant_topic')['sentiment'].mean().reset_index()

# Add description label for visualization
topic_sentiment['sentiment_label'] = topic_sentiment['sentiment'].apply(sentiment_description)

# Create interactive bar plot using Plotly
fig = px.bar(topic_sentiment, x='dominant_topic', y='sentiment',
             title="Average Sentiment by Topic",
             labels={'dominant_topic': 'Topic', 'sentiment': 'Average Sentiment'},
             color='sentiment', color_continuous_scale='RdYlGn',
             hover_data=['sentiment_label'])  # Show description on hover

# Show the plot
fig.show()

In [ ]:
import matplotlib.pyplot as plt

# Lexicon-based frame detection using keywords in the topic. Source: Wikipedia
frame_keywords = {
    'economic': [
        'tax', 'inflation', 'budget', 'cost', 'price', 'market', 'employment', 'wage', 'gdp', 'trade', 'recession'
    ],
    'moral': [
        'justice', 'fair', 'right', 'wrong', 'moral', 'ethics', 'duty', 'virtue', 'integrity', 'values'
    ],
    'security': [
        'threat', 'violence', 'risk', 'terror', 'security', 'crime', 'military', 'conflict', 'attack', 'protection'
    ],
    'political': [
        'government', 'policy', 'party', 'election', 'vote', 'congress', 'president', 'law', 'republican', 'democrat'
    ],
    'health': [
        'disease', 'virus', 'vaccine', 'pandemic', 'hospital', 'mental', 'therapy', 'nutrition', 'healthcare', 'fitness'
    ],
    'environmental': [
        'climate', 'pollution', 'emissions', 'carbon', 'conservation', 'wildlife', 'recycling', 'green', 'sustainability', 'global warming'
    ],
    'technological': [
        'ai', 'artificial intelligence', 'algorithm', 'data', 'cyber', 'robot', 'machine', 'tech', 'startup', 'innovation'
    ],
    'educational': [
        'school', 'college', 'university', 'student', 'curriculum', 'education', 'teacher', 'literacy', 'learning', 'exam'
    ],
    'racial_gender': [
        'race', 'racism', 'gender', 'inequality', 'feminism', 'diversity', 'inclusion', 'identity', 'sexism', 'equity'
    ],
    'media': [
        'journalist', 'press', 'coverage', 'bias', 'fake news', 'report', 'headline', 'broadcast', 'media outlet', 'censorship'
    ],
    'beauty': [
        'makeup', 'skincare', 'cosmetic', 'fashion', 'style', 'glamour', 'appearance', 'hair', 'aesthetic', 'model'
    ],
    'science': [
        'research', 'study', 'scientist', 'experiment', 'theory', 'discovery', 'physics', 'biology', 'chemistry', 'lab'
    ],
    'cultural': [
        'tradition', 'heritage', 'art', 'music', 'film', 'festival', 'language', 'custom', 'ceremony', 'belief'
    ],
    'legal': [
        'court', 'judge', 'verdict', 'lawsuit', 'trial', 'justice', 'legal', 'sentence', 'appeal', 'constitution'
    ],
    'religious': [
        'faith', 'god', 'church', 'muslim', 'christian', 'bible', 'religion', 'worship', 'spiritual', 'doctrine'
    ],
    'international': [
        'foreign', 'diplomacy', 'treaty', 'alliance', 'embassy', 'sanction', 'global', 'international', 'relations', 'border'
    ],
    'social': [
        'community', 'welfare', 'inequality', 'poverty', 'family', 'housing', 'benefits', 'support', 'neighborhood', 'charity'
    ],
    'sports': [
        'game', 'team', 'player', 'match', 'tournament', 'league', 'score', 'coach', 'win', 'championship'
    ]
}

def detect_frames(text):
    frames = []
    for key, words in frame_keywords.items():
        if any(word in text.lower() for word in words):
            frames.append(key)
    return frames

df['frames'] = df['article_text'].apply(detect_frames)

# Count frames per article
df_frame_counts = df['frames'].apply(lambda x: len(x)).value_counts().reset_index()
df_frame_counts.columns = ['Number of Frames', 'Frequency']

# Create an interactive bar plot using Plotly
fig = px.bar(df_frame_counts, x='Number of Frames', y='Frequency',
             title="Number of Frames per Article",
             labels={'Number of Frames': 'Number of Frames', 'Frequency': 'Frequency'},
             color='Frequency', color_continuous_scale='Viridis')

# Show the plot
fig.show()

In [ ]:
import spacy
from collections import Counter

# Load spaCy's model for NER
nlp = spacy.load("en_core_web_sm")

# Named entity recognition
def get_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    return entities

df['entities'] = df['article_text'].apply(get_entities)

# Count entity occurrences
entity_counts = Counter([entity for sublist in df['entities'] for entity in sublist])
entity_counts.most_common(10)

[('Trump', 49283),
 ('Biden', 34250),
 ('first', 22779),
 ('one', 18995),
 ('Harris', 16239),
 ('two', 15534),
 ('U.S.', 13290),
 ('US', 12007),
 ('Donald Trump', 11931),
 ('Democrats', 10183)]

In [ ]:
import plotly.express as px

fig = px.histogram(
    df,
    x='dominant_topic',
    title="Topic Distribution in Articles",
    labels={'dominant_topic': 'Topic'},
    nbins=len(df['dominant_topic'].unique())
)
fig.update_layout(
    xaxis_title="Topic",
    yaxis_title="Article Count"
)
fig.show()

In [ ]:
import plotly.graph_objects as go

# Group articles by `dominant_topic` and calculate frame frequencies
topic_frame_analysis = df.groupby('dominant_topic')['frames'].apply(lambda x: x.explode().value_counts()).unstack().fillna(0)

# Prepare data for Plotly stacked bar chart
fig = go.Figure()
for frame in topic_frame_analysis.columns:
    fig.add_trace(
        go.Bar(
            x=topic_frame_analysis.index,
            y=topic_frame_analysis[frame],
            name=frame
        )
    )

fig.update_layout(
    barmode='stack',
    title="Frame Distribution by Topic",
    xaxis_title="Topic",
    yaxis_title="Frequency of Frames"
)
fig.show()


In [ ]:
import plotly.express as px

# Visualize the political bias distribution using Plotly
bias_count = df['nlp_label'].value_counts().reset_index()
bias_count.columns = ['Bias', 'Article Count']

fig_bias = px.bar(bias_count, x='Bias', y='Article Count',
                  title="Political Bias Distribution",
                  labels={'Bias': 'Political Bias', 'Article Count': 'Article Count'},
                  color='Article Count', color_continuous_scale='RdYlGn')
fig_bias.show()

# Frame distribution by political leaning using Plotly
bias_frame_analysis = df.groupby('nlp_label')['frames'].apply(lambda x: x.explode().value_counts()).unstack().fillna(0)

# Reshape the DataFrame for plotting
bias_frame_analysis = bias_frame_analysis.reset_index().melt(id_vars='nlp_label', var_name='Frame', value_name='Frequency')
bias_frame_analysis.columns = ['Political Bias', 'Frame', 'Frequency']

fig_frame = px.bar(bias_frame_analysis, x='Political Bias', y='Frequency', color='Frame',
                   title="Frame Distribution by Political Bias",
                   labels={'Political Bias': 'Political Bias', 'Frequency': 'Frequency of Frames'},
                   barmode='stack', color_discrete_sequence=px.colors.qualitative.Set1)
fig_frame.show()

In [ ]:
# Save output as a csv file
df.to_csv('/content/drive/MyDrive/IS450 Group 5/2025 datasets/Classified_articles_with_political_bias.csv', index=False)

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the pre-trained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bucketresearch/politicalBiasBERT")
model = AutoModelForSequenceClassification.from_pretrained("bucketresearch/politicalBiasBERT")

# Function to classify the political bias of a single text
def classify_political_bias(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Get model outputs
    outputs = model(**inputs)
    logits = outputs.logits

    # Apply softmax to get probabilities for each class
    probabilities = torch.softmax(logits, dim=-1)[0].tolist()

    # Return the predicted class and probabilities
    # Predicted class: 0 -> Left, 1 -> Center, 2 -> Right
    predicted_class = torch.argmax(logits, dim=-1).item()
    return predicted_class, probabilities

# Apply the classification function to the 'article_text' column
df[['political_bias', 'bias_probabilities']] = df['article_text'].apply(lambda x: pd.Series(classify_political_bias(x)))

# Map numerical bias labels to their respective meanings
df['political_bias'] = df['political_bias'].map({0: 'Left', 1: 'Center', 2: 'Right'})

# Show the updated DataFrame
print(df[['unique_id', 'headline', 'outlet', 'political_bias', 'bias_probabilities']].head())

In [ ]:
=# Save output as a csv file
df.to_csv('/content/drive/MyDrive/zara/classified_articles_with_political_bias_WITH_BERT.csv', index=False)

# Model Evaluation

In [4]:
# Read the generated csv
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/IS450 Group 5/2025 datasets/Classified_articles_with_political_bias.csv')
df.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label,dominant_topic,sentiment,frames,entities,machine_political_bias,bias_probabilities,sentiment_label,human_annotation
0,9f02710888,National Review,The Latest Biden-Harris Madness: Funding DEI I...,https://www.nationalreview.com,The latest Biden-Harris proposed budget wants ...,images/9f02710888.jpg,Likely to be Bias,Likely to be Unbiased,0,0.174219,"['economic', 'moral', 'security', 'political',...","['Biden-Harris', 'an additional $17 million', ...",Center,"[0.14993520081043243, 0.5630002617835999, 0.28...",Neutral,Right
1,7381c9ee41,The Globe and Mail,Liberal MPs say Quebec French-language health ...,https://www.theglobeandmail.com,Two Montreal-area Liberal MPs are raising conc...,images/7381c9ee41.jpg,Likely to be Bias,Likely to be Bias,0,0.028386,"['moral', 'political', 'health', 'technologica...","['Two', 'Montreal', 'Quebec', 'French', 'last ...",Center,"[0.13426877558231354, 0.5512582659721375, 0.31...",Neutral,Center
2,cf74f338b4,Forbes,UAW Hopes To Expand Its Ranks As Alabama Merce...,https://www.forbes.com,"More than 5,000 workers at two Mercedes-Benz p...",images/cf74f338b4.jpg,Likely to be Unbiased,Likely to be Unbiased,0,0.061828,"['economic', 'security', 'political', 'health'...","['More than 5,000', 'two', 'Mercedes-Benz', 'A...",Center,"[0.047266799956560135, 0.913899838924408, 0.03...",Neutral,Left
3,fcf1ade5c5,Financial Times,US consumer spending slowdown weighs on travel...,https://www.ft.com,US consumers are reining in spending on travel...,images/fcf1ade5c5.jpg,Likely to be Bias,Likely to be Bias,20,0.053401,"['economic', 'political', 'health', 'technolog...","['US', 'Disney', 'Hilton', 'this week', 'Ameri...",Center,"[0.26279398798942566, 0.4689684808254242, 0.26...",Neutral,Center
4,ec1844ce20,The Guardian,UK needs system for recording AI misuse and ma...,https://www.theguardian.com,The UK needs a system for recording misuse and...,images/ec1844ce20.jpg,Likely to be Bias,Likely to be Unbiased,0,0.002976,"['economic', 'moral', 'security', 'political',...","['UK', 'AI', 'UK', 'the Centre for Long-Term R...",Center,"[0.205074280500412, 0.6088244915008545, 0.1861...",Neutral,Center


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   unique_id               150 non-null    object 
 1   outlet                  150 non-null    object 
 2   headline                150 non-null    object 
 3   url                     150 non-null    object 
 4   article_text            150 non-null    object 
 5   image                   150 non-null    object 
 6   nlp_label               150 non-null    object 
 7   nlp-image_label         150 non-null    object 
 8   dominant_topic          150 non-null    int64  
 9   sentiment               150 non-null    float64
 10  frames                  150 non-null    object 
 11  entities                150 non-null    object 
 12  machine_political_bias  150 non-null    object 
 13  bias_probabilities      150 non-null    object 
 14  sentiment_label         150 non-null    ob

In [ ]:
# Generate a random sample to annotate
import pandas as pd

def balanced_sample(df, label_col='political_bias', n_per_class=50, random_state=42):
    """
    Randomly sample a balanced number of rows per class from the political_bias column.

    Args:
        df (pd.DataFrame): Your full dataframe.
        label_col (str): Column name with class labels.
        n_per_class (int): Number of samples per class.
        random_state (int): Seed for reproducibility.

    Returns:
        pd.DataFrame: Balanced sampled DataFrame.
    """
    sampled_df = (
        df.groupby(label_col, group_keys=False)
          .apply(lambda x: x.sample(min(len(x), n_per_class), random_state=random_state))
          .reset_index(drop=True)
    )
    return sampled_df

# Sample a balanced dataset
sampled_df = balanced_sample(df, label_col='political_bias', n_per_class=50)

# Save it to CSV for annotation
sampled_df.to_csv("/content/drive/MyDrive/IS450 Group 5/politicalbiasbert/political_bias_annotation_sample.csv", index=False)

print("Sample saved to 'political_bias_annotation_sample.csv'")

✅ Sample saved to 'political_bias_annotation_sample.csv'


<ipython-input-7-58342c84623e>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(min(len(x), n_per_class), random_state=random_state))


In [3]:
# Open the annotated csv
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/IS450 Group 5/politicalbiasbert/political_bias_annotation_sample - political_bias_annotation_sample - political_bias_annotation_sample - political_bias_annotation_sample.csv')
df.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label,dominant_topic,sentiment,frames,entities,machine_political_bias,bias_probabilities,sentiment_label,human_annotation
0,9f02710888,National Review,The Latest Biden-Harris Madness: Funding DEI I...,https://www.nationalreview.com,The latest Biden-Harris proposed budget wants ...,images/9f02710888.jpg,Likely to be Bias,Likely to be Unbiased,0,0.174219,"['economic', 'moral', 'security', 'political',...","['Biden-Harris', 'an additional $17 million', ...",Center,"[0.14993520081043243, 0.5630002617835999, 0.28...",Neutral,Right
1,7381c9ee41,The Globe and Mail,Liberal MPs say Quebec French-language health ...,https://www.theglobeandmail.com,Two Montreal-area Liberal MPs are raising conc...,images/7381c9ee41.jpg,Likely to be Bias,Likely to be Bias,0,0.028386,"['moral', 'political', 'health', 'technologica...","['Two', 'Montreal', 'Quebec', 'French', 'last ...",Center,"[0.13426877558231354, 0.5512582659721375, 0.31...",Neutral,Center
2,cf74f338b4,Forbes,UAW Hopes To Expand Its Ranks As Alabama Merce...,https://www.forbes.com,"More than 5,000 workers at two Mercedes-Benz p...",images/cf74f338b4.jpg,Likely to be Unbiased,Likely to be Unbiased,0,0.061828,"['economic', 'security', 'political', 'health'...","['More than 5,000', 'two', 'Mercedes-Benz', 'A...",Center,"[0.047266799956560135, 0.913899838924408, 0.03...",Neutral,Center
3,fcf1ade5c5,Financial Times,US consumer spending slowdown weighs on travel...,https://www.ft.com,US consumers are reining in spending on travel...,images/fcf1ade5c5.jpg,Likely to be Bias,Likely to be Bias,20,0.053401,"['economic', 'political', 'health', 'technolog...","['US', 'Disney', 'Hilton', 'this week', 'Ameri...",Center,"[0.26279398798942566, 0.4689684808254242, 0.26...",Neutral,Center
4,ec1844ce20,The Guardian,UK needs system for recording AI misuse and ma...,https://www.theguardian.com,The UK needs a system for recording misuse and...,images/ec1844ce20.jpg,Likely to be Bias,Likely to be Unbiased,0,0.002976,"['economic', 'moral', 'security', 'political',...","['UK', 'AI', 'UK', 'the Centre for Long-Term R...",Center,"[0.205074280500412, 0.6088244915008545, 0.1861...",Neutral,Center


In [4]:
from sklearn.metrics import cohen_kappa_score

# Calculate Cohen's Kappa between 'human_annotation' and 'machine_political_bias'
kappa = cohen_kappa_score(df['human_annotation'], df['machine_political_bias'])

print(f"Cohen's Kappa: {kappa}")

Cohen's Kappa: 0.83
